# Deelstudie 1: Retrospectieve analyse van historische YouTube-data - Relatie tussen het aantal comments en het aantal views

**Hypothese:** Video’s met een hoger aantal comments vertonen een significant hoger aantal views

**Manier van werken:**
- Beschrijvende statistiek
- Normaliteitstoets
- Correlatieanalyse
- Lineaire regressieanalyse
- Multivariate regressieanalyse

In [ ]:
# Import libraries
import math
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
# General settings
sns.set_theme(palette='muted')

In [ ]:
# Import data
videos = pd.read_excel('../data/videos.xlsx', index_col=0)
videos

In [ ]:
# Get info about the data
print(videos.shape) # rows, columns
print(videos.dtypes) # column names and data type

## STAP 1: Beschrijvende statistiek

### 1.1 Verkenning van de data

In [ ]:
# Set engagement metrics
engagement_metrics = ['comments', 'views', 'shares', 'likes', 'dislikes', 'engagement']

In [ ]:
# describe the engagement metrics
videos[engagement_metrics].describe().T

In [ ]:
# Visualize engagement metrics distribution

# Determine number of rows and columns for subplots
n_metrics = len(engagement_metrics)
n_cols = 2
n_rows = math.ceil(n_metrics / n_cols)

# Create plots and subplots
fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 4 * n_rows))
for i, metric in enumerate(engagement_metrics):
    row, col = divmod(i, n_cols)
    sns.histplot(videos[metric], kde=True, ax=axes[row, col])
    axes[row, col].set_title(f'Distributie van {metric}')
    axes[row, col].set_ylabel('Frequentie')
    axes[row, col].set_xlabel(metric.capitalize())
    axes[row, col].xaxis.set_major_formatter(mtick.FuncFormatter(lambda x, _: f'{int(x):,}'))
# Disable empty subplots if any
if n_metrics % n_cols != 0:
    axes[-1, -1].axis('off')
# Render plots
plt.tight_layout()
plt.show()

In [ ]:
# Visualize engagement metrics boxplots


# Determine number of rows and columns for subplots
n_metrics = len(engagement_metrics)
n_cols = 2
n_rows = math.ceil(n_metrics / n_cols)

# Create plots and subplots
fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 4 * n_rows))
for i, metric in enumerate(engagement_metrics):
    row, col = divmod(i, n_cols)
    sns.boxplot(x=videos[metric], ax=axes[row, col])
    axes[row, col].set_title(f'Boxplot van {metric}')
    axes[row, col].set_xlabel(metric.capitalize())
    axes[row, col].xaxis.set_major_formatter(mtick.FuncFormatter(lambda x, _: f'{int(x):,}'))
# Disable empty subplots if any
if n_metrics % n_cols != 0:
    axes[-1, -1].axis('off')
# Render plots
plt.tight_layout()
plt.show()

## STAP 2: Normaliteitstoets

In [ ]:
# Normality test with Shapiro-Wilk test
shapiro_comments = stats.shapiro(videos['comments'])
shapiro_views = stats.shapiro(videos['views'])

# Comments
print(f'Shapiro-Wilk test aantal comments:')
print(f'  W-statistic: {shapiro_comments.statistic:.4f}')
print(f'  p-value: {shapiro_comments.pvalue:.4e}')
print(f'Skewness comments: {videos["comments"].skew():.4f}')

# Views
print(f'\nShapiro-Wilk test aantal views:')
print(f'  W-statistic: {shapiro_views.statistic:.4f}')
print(f'  p-value: {shapiro_views.pvalue:.4e}')
print(f'Skewness views: {videos["views"].skew():.4f}')

### Resultaten en interpretatie normaliteitstoets

De Shapiro-Wilk test toonde het volgende resultaat:

- **Comments:**
    - W-statistic: 0.3718
    - p-value: 4.4922e-38
    - Skewness: 6.6380

- **Views:**
    - W-statistic: 0.3184
    - p-value: 3.5041e-39
    - Skewness: 8.4351

**Interpretatie**

Voor beide variabelen (`comments` en `views`) is de p-waarde < 0.05, wat betekent dat de nulhypothese van normaliteit wordt verworpen. Bovendien tonen de skewness-waarden een sterke rechtsscheefheid (>6), wat wijst op een ernstige afwijking van normaliteit. Dit bevestigt de noodzaak om een log-transformatie toe te passen alvorens correlatie- en regressieanalyses uit te voeren, conform de richtlijnen van `Field (2018)`.

In [ ]:
# Log-transformation
videos['comments_log'] = np.log1p(videos['comments'])
videos['views_log'] = np.log1p(videos['views'])

# Redo Shapiro-Wilk test with log-transformed data
shapiro_comments_log = stats.shapiro(videos['comments_log'])
shapiro_views_log = stats.shapiro(videos['views_log'])

# Comments log
print(f'Shapiro-Wilk test log-comments:')
print(f'  W-statistic: {shapiro_comments_log.statistic:.4f}')
print(f'  p-value: {shapiro_comments_log.pvalue:.4e}')
print(f'Skewness log-comments: {videos["comments_log"].skew():.4f}')

# Views log
print(f'\nShapiro-Wilk test log-views:')
print(f'  W-statistic: {shapiro_views_log.statistic:.4f}')
print(f'  p-value: {shapiro_views_log.pvalue:.4e}')
print(f'Skewness log-views: {videos["views_log"].skew():.4f}')

In [ ]:
# Visualize log-transformed data

# Prepare plots
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# --- Distribution plots on top ---
# Log-comments
sns.histplot(videos['comments_log'], kde=True, ax=axes[0, 0])
axes[0, 0].set_title('Distributie log-getransformeerde comments')
axes[0, 0].set_xlabel('Log(Comments)')
axes[0, 0].set_ylabel('Frequentie')

# Log-views
sns.histplot(videos['views_log'], kde=True, ax=axes[0, 1])
axes[0, 1].set_title('Distributie log-getransformeerde Views')
axes[0, 1].set_xlabel('Log(Views)')
axes[0, 1].set_ylabel('Frequentie')

# --- Boxplots at the bottom ---
# Log-comments boxplot
sns.boxplot(x=videos['comments_log'], ax=axes[1, 0])
axes[1, 0].set_title('Boxplot log-getransformeerde comments')
axes[1, 0].set_xlabel('Log(Comments)')
axes[1, 0].set_ylabel('')

# Log-views boxplot
sns.boxplot(x=videos['views_log'], ax=axes[1, 1])
axes[1, 1].set_title('Boxplot log-getransformeerde views')
axes[1, 1].set_xlabel('Log(Views)')
axes[1, 1].set_ylabel('')

# Render plots
plt.tight_layout()
plt.show()

### Resultaten en interpretatie na log-transformatie

Na het toepassen van een log-transformatie op de variabelen `comments` en `views` werden opnieuw de Shapiro-Wilk tests en skewness-berekeningen uitgevoerd om de verdelingen te evalueren.

**Samenvatting van de resultaten:**

- **Log-comments:**
    - W-statistic: 0.9720
    - p-value: 3.9275e-08
    - Skewness: 0.4462

- **Log-views:**
    - W-statistic: 0.9464
    - p-value: 2.1011e-12
    - Skewness: -0.8047

**Interpretatie**

De resultaten tonen aan dat, hoewel de Shapiro-Wilk test nog steeds significant blijft (p < 0.05), de skewness-waarden aanzienlijk zijn verbeterd en zich nu bevinden binnen het interval [-1, +1]. Dit wijst op een acceptabele mate van normaliteit (Field, 2018). Visuele inspectie via histogrammen bevestigt dat de verdelingen van zowel `log-comments` als `log-views` duidelijk symmetrischer en dichter bij een normale verdeling liggen in vergelijking met de oorspronkelijke data. Ook de boxplots tonen een meer evenwichtige spreiding van de data, met minder extreme uitschieters.

Op basis van deze bevindingen kan besloten worden dat de log-getransformeerde variabelen voldoende normaal verdeeld zijn om verder te gaan met de Pearson-correlatie en lineaire regressieanalyse. Het blijft echter aangewezen om naast Pearson ook Spearman's rangcorrelatie te rapporteren, aangezien de oorspronkelijke variabelen sterk afweken van normaliteit.

## STAP 3: Correlatieanalyse

In [ ]:
# Pearson-correlation on log-transformed variables
r, p_value = stats.pearsonr(videos['comments_log'], videos['views_log'])
print(f'Pearson-correlatie: {r:.4f}')
print(f'p-waarde: {p_value:.4e}')

# Spearman-correlatie on log-transformed variables
r_spearman, p_value_spearman = stats.spearmanr(videos['comments_log'], videos['views_log'])
print(f'Spearman-correlatie: {r_spearman:.4f}')
print(f'p-waarde: {p_value_spearman:.4e}')

In [ ]:
# Scatterplot for correlation analysis
sns.regplot(
    x='comments_log',
    y='views_log',
    data=videos,
    scatter_kws={'alpha': 0.5, 'color': '#4C72B0'},  # Data points
    color='#C44E52',  # Trend Line and Confidence Interval
    line_kws={'alpha': 1}  # Trend line transparency
)
plt.title('Scatterplot met regressielijn: Log(Comments) versus Log(Views)')
plt.xlabel('Log(Comments)')
plt.ylabel('Log(Views)')
plt.show()

### Resultaten en interpretatie correlatieanalyse

**Pearson-correlatie:**
- Coëfficiënt: 0.7517
- Betekenis: Er is een sterke positieve lineaire samenhang tussen het aantal comments en het aantal views (beide log-getransformeerd). Dit wijst erop dat video's met meer comments gemiddeld ook meer views behalen.

**Spearman-correlatie:**
- Coëfficiënt: 0.7613
- Betekenis: De Spearman-correlatie bevestigt een sterke positieve monotone relatie tussen het aantal comments en views, wat betekent dat een toename van comments gepaard gaat met een toename van views, ongeacht de exacte lineaire vorm.

**Interpretatie**

Beide correlatiecoëfficiënten wijzen op een sterke positieve relatie tussen het aantal comments en het aantal views, waarmee de hypothese ondersteund wordt. De Pearson-coëfficiënt van 0.75 wijst op een substantiële lineaire samenhang, terwijl de Spearman-coëfficiënt van 0.76 het bestaan van een robuuste monotone relatie bevestigt. Dit impliceert dat video's die meer engagement genereren in de vorm van comments, ook aanzienlijk meer kijkers aantrekken.

Het scatterplot met regressielijn bevestigt dit patroon visueel en toont een duidelijke positieve trend. Hoewel er enkele spreiding en outliers zichtbaar zijn, blijft het algemene verband sterk aanwezig. Deze bevindingen zijn in lijn met bestaande literatuur die het belang van engagement benadrukt in het vergroten van bereik op sociale mediaplatformen `(Covington et al., 2016)`.

## STAP 4: Lineaire regressieanalyse

In [ ]:
# Define X and y
X = videos[['comments_log']] # Independent variable
y = videos['views_log'] # Dependent variable

# Add constant to the model
X = sm.add_constant(X)

# Execute regression
model = sm.OLS(y, X).fit()
model.summary()

### Resultaten en interpretatie eenvoudige lineaire regressie

De regressieanalyse werd uitgevoerd met `log-comments` als onafhankelijke variabele en `log-views` als afhankelijke variabele.

**Samenvatting van de resultaten:**
- Coëfficiënt (slope): 0.8048
- Intercept: 6.9895
- R²: 0.565
- p-waarde coëfficiënt: < 0.001

**Interpretatie**

De regressieanalyse toont aan dat het aantal comments een significante voorspeller is van het aantal views (beide log-getransformeerd). De coëfficiënt van 0.8048 wijst erop dat een stijging van 1 log-eenheid in het aantal comments geassocieerd is met een stijging van ongeveer 0.80 log-eenheden in het aantal views. Het model verklaart 56,5% van de variantie in het aantal views (R² = 0.565), wat duidt op een substantieel effect.

De p-waarde (< 0.001) bevestigt dat het effect statistisch significant is. Deze bevinding ondersteunt de hypothese en bevestigt dat engagement via comments een belangrijke rol speelt in het vergroten van het bereik van video's. Dit resultaat is in lijn met bestaande literatuur die het belang van gebruikersinteractie onderstreept in het YouTube-algoritme `(Covington et al., 2016)`.

Hoewel het model sterk is, geven de diagnostische tests (zoals de skewness en kurtosis van de residuen) aan dat er nog enige afwijking van perfect normaliteit is. Dit is te verwachten bij grote datasets met socialemediadata en vormt geen ernstig probleem voor de robuustheid van het model `(Field, 2018)`.

## STAP 5: Multivariate regressieanalyse

In [ ]:
# Log-transform engagement metrics that haven't been transformed yet
# Skip engagement metric 'engagement' as it is not a raw count but the result of a formula
videos['likes_log'] = np.log1p(videos['likes'])
videos['dislikes_log'] = np.log1p(videos['dislikes'])
videos['shares_log'] = np.log1p(videos['shares'])

In [ ]:
# Define dependent variable and independent variables
X_multi = videos[['comments_log', 'likes_log', 'shares_log', 'dislikes_log']]
y_multi = videos['views_log']

# Add constant to the model
X_multi = sm.add_constant(X_multi)

# Execute regression
multi_model = sm.OLS(y_multi, X_multi).fit()
multi_model.summary()

In [ ]:
# VIF-test (check multicollinearity)
vif_data = pd.DataFrame()
vif_data['Feature'] = X_multi.columns
vif_data['VIF'] = [variance_inflation_factor(X_multi.values, i) for i in range(X_multi.shape[1])]
vif_data

### Resultaten en interpretatie multivariate regressie

De multivariate regressie werd uitgevoerd met `log-comments`, `log-likes` en `log-shares` als onafhankelijke variabelen en `log-views` als afhankelijke variabele.

**Samenvatting van de resultaten:**
- Coëfficiënt (log-comments): 0.0306 (p = 0.200)
- Coëfficiënt (log-likes): 0.7023 (p < 0.001)
- Coëfficiënt (log-shares): 0.3121 (p < 0.001)
- R²: 0.904
- VIF-waarden:
    - comments_log: 2.55
    - likes_log: 4.61
    - shares_log: 4.00

**Interpretatie**

Het model verklaart 90.4% van de variantie in het aantal views, wat wijst op een uitstekende voorspellende kracht. De VIF-waarden tonen aan dat er geen problematische multicollineariteit aanwezig is.

De resultaten tonen dat zowel likes als shares sterke en significante voorspellers zijn van het aantal views. De coëfficiënt van `log-likes` (0.7023) wijst erop dat een stijging van 1 log-eenheid in likes geassocieerd is met een stijging van ~0.70 log-eenheden in views, terwijl een stijging in shares (coëfficiënt = 0.3121) eveneens een significante bijdrage levert.

Opvallend is dat de invloed van comments op views, die in de eenvoudige regressie nog sterk was, wegvalt zodra gecontroleerd wordt voor likes en shares (p = 0.200). Dit suggereert dat het aanvankelijke effect van comments deels wordt verklaard door hun samenhang met andere engagement metrics.

Deze bevinding nuanceert de hypothese: hoewel comments initieel een significante voorspeller lijken, verdwijnt dit effect zodra andere vormen van engagement in rekening worden gebracht. Dit onderstreept het belang van een multivariate aanpak en bevestigt dat likes en shares cruciale determinanten zijn van het bereik van YouTube-video's `(Covington et al., 2016)`.

### Residucontrole

In [ ]:
# QQ-plot of the residues (multivariate regressie)
fig = sm.qqplot(multi_model.resid, line='45')
plt.title('QQ-plot van de residuen (multivariate regressie)')
plt.xlabel('Theoretische kwantielen')
plt.ylabel('Waargenomen kwantielen')
plt.show()

In [ ]:
sns.residplot(
    x=multi_model.fittedvalues,
    y=multi_model.resid,
    lowess=True,
    line_kws={'color': 'red'}
)
plt.xlabel('Voorspelde waarden')
plt.ylabel('Residuen')
plt.title('Voorspelde waarden vs. residuen (multivariate regressie)')
plt.show()

### Residucontrole

Om de aannames van de lineaire regressie te toetsen, werd een residucontrole uitgevoerd.

- **QQ-plot:** De QQ-plot laat zien dat de residuen grotendeels de diagonaal volgen, wat wijst op een redelijk normale verdeling van de residuen. Er zijn lichte afwijkingen zichtbaar aan de uiteinden (staarten), wat verwacht kan worden bij socialemediadata en geen ernstige bedreiging vormt voor de validiteit van het model (Field, 2018).

- **Voorspelde waarden vs. residuen:** De plot toont geen duidelijke systematische patronen, wat suggereert dat de homoscedasticiteitsaanname (gelijke spreiding van residuen) voldoende is voldaan. De residuen zijn willekeurig verspreid rond de nul-lijn, wat bevestigt dat het model geen grote structurele fouten bevat.

Op basis van deze controle kan geconcludeerd worden dat het regressiemodel robuust genoeg is en dat de belangrijkste aannames in voldoende mate zijn vervuld om betrouwbare inferenties te maken.